In [2]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("pravar_sharma_updated.pdf")
docs = loader.load()

In [3]:
docs

[Document(metadata={'producer': 'Skia/PDF m137', 'creator': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/137.0.0.0 Safari/537.36', 'creationdate': 'D:20250615101126', 'title': 'first resume', 'moddate': 'D:20250615101126', 'author': 'first resume', 'keywords': 'Python, Django, FastAPI\nJavaScript, React, Vite, Tailwind CSS\nREST APIs, PostgreSQL, MySQL\nAWS (EC2, S3, Lambda, SQS, IAM)\nDocker, GitHub Actions, CI/CD\nMicroservices Architecture\nSystem Design, Agile/Scrum\nGenerative AI Integration (LangChain, OpenAI API)', 'source': 'pravar_sharma_updated.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Pravar Sharma\nJaipur,India pravar.sharma@gmail.com+91 9414472171 in/pravar-sharma-3410a199\nSUMMARY\nResults-driven Backend Developer with 5+ years of experience designing and developing scalable web applications using Python, Django,\nReact, and AWS. Proven track record in building SaaS platforms, integrating third-party API

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
documents = text_splitter.split_documents(docs)

In [7]:
documents

[Document(metadata={'producer': 'Skia/PDF m137', 'creator': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/137.0.0.0 Safari/537.36', 'creationdate': 'D:20250615101126', 'title': 'first resume', 'moddate': 'D:20250615101126', 'author': 'first resume', 'keywords': 'Python, Django, FastAPI\nJavaScript, React, Vite, Tailwind CSS\nREST APIs, PostgreSQL, MySQL\nAWS (EC2, S3, Lambda, SQS, IAM)\nDocker, GitHub Actions, CI/CD\nMicroservices Architecture\nSystem Design, Agile/Scrum\nGenerative AI Integration (LangChain, OpenAI API)', 'source': 'pravar_sharma_updated.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Pravar Sharma\nJaipur,India pravar.sharma@gmail.com+91 9414472171 in/pravar-sharma-3410a199\nSUMMARY\nResults-driven Backend Developer with 5+ years of experience designing and developing scalable web applications using Python, Django,\nReact, and AWS. Proven track record in building SaaS platforms, integrating third-party API

In [11]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(documents, OllamaEmbeddings(model="llama3.2"))

In [13]:
query = "RESTful APIs for booking, cancellation"
result = db.similarity_search(query)
result[0].page_content

'Docker, GitHub Actions, CI/CD•\nMicroservices Architecture•\nSystem Design, Agile/Scrum•\nGenerative AI Integration (LangChain, OpenAI API)•'

In [22]:
from langchain_ollama import OllamaLLM
llm = OllamaLLM(model="llama3.2")
llm

OllamaLLM(model='llama3.2')

In [23]:
# design Chatprompt template
from langchain_core.prompts import ChatPromptTemplate
# context is all the documents there are in the vector Database.
# input is the question I am asking.
prompt = ChatPromptTemplate.from_template(
    """
    Answer the following question based only on the provided context.
    Think step by step before providing answer.
    <context>
    {context}
    </context>
    Question: {input}
    """
)

In [24]:
# create stuff document chain
from langchain.chains.combine_documents import create_stuff_documents_chain

doc_chain = create_stuff_documents_chain(llm, prompt)

In [25]:
# Retrivers
"""
It is an interface that return the document given an unstructured query, it is more like a catalog that finds the index 
if the query you have asked and then find it in the vector db, and also helps in narrow down the search.
"""

retriver = db.as_retriever()
retriver

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x1252e20c0>, search_kwargs={})

In [26]:
# Retrival Chain
"""
When we combine retriver anc document chain together it is known as retrival chain.
This chain takes user input, which is then pass to retriver to fetch relevent documents.
This combined is pass to an LLM to generate response.
"""

from langchain.chains import create_retrieval_chain
retrival_chain = create_retrieval_chain(retriver, doc_chain)


In [34]:
response = retrival_chain.invoke({"input": "I just wanted to know what is is expert on? based on the project he has mentioned not just the skills he mentioned."""})

In [35]:
response['answer']

"Based on the provided context, it seems that Pravar Sharma is an expert in designing and developing scalable web applications using Python, Django, React, and AWS. He has a proven track record of building SaaS platforms, integrating third-party APIs, and deploying secure, cloud-native apps.\n\nSpecifically, his experience in microservices architecture, system design, Agile/Scrum, and CI/CD pipelines suggests that he is well-versed in designing and developing complex, scalable systems. His work on Generative AI integration (LangChain, OpenAI API) indicates that he is also interested in exploring the potential of AI to enhance intelligent application capabilities.\n\nIn terms of specific areas where he has demonstrated expertise, it appears that Pravar Sharma has a strong background in:\n\n1. Building scalable RESTful APIs and microservice-based backend modules.\n2. Integrating third-party APIs and deploying secure, cloud-native apps.\n3. Optimizing database queries and indexing for hig